In [ ]:
#Importar librerias
import pandas as pd
import requests
import json
import time
import os
from Geolocalizacion.concatenacion_qr.qr_generation import coords_col, generate_qrs, generate_urls  
import geopandas as gpd

In [ ]:
def find_lista():
    try:
        lista = pd.read_csv(r"", sep=',', header=None)
    except: 
        
        print('El path no contiene lista')
    return(lista[0])

In [ ]:
def set_key(base,j,dir_entry_path,lista=find_lista(),key=find_lista()[0]):
    i=0
    key=key
    url = "https://maps.googleapis.com/maps/api/geocode/json?address="+str('Calle ') +str(base.Domicilio[j])+"&key="+str(key)+"&language=es-419&region=.mx"
    print(url)
    try:
        respon = requests.get(url)
        print(base.INTFIS_RS_FD_DIRECCION[j])           
        print(json.loads(respon.text)['status'])
        if(json.loads(respon.text)['status']=='ZERO_RESULTS'):
              respon = {}
        else:
            while (json.loads(respon.text)['status']!='OK'):
                key_n=lista[i+1]
                url = "https://maps.googleapis.com/maps/api/geocode/json?address="+str(base.INTFIS_RS_FD_DIRECCION[j])+"&key="+str(key_n)+"&language=es-419&region=.mx"
                respon = requests.get(url)
                i=i+1
                print(json.loads(respon.text)['status'])
                print(base.INTFIS_RS_FD_DIRECCION[j])
    except TimeoutError: 
        print('Timeout')
        time.sleep(10)
        url = "https://maps.googleapis.com/maps/api/geocode/json?address="+str(base.INTFIS_RS_FD_DIRECCION[j])+"&key="+str(key)+"&language=es-419&region=.mx"
        respon = requests.get(url)
        print(base.INTFIS_RS_FD_DIRECCION[j])
        print(json.loads(respon.text)['status'])
    except:
        pass

    return(respon,base.Domicilio[j])

In [ ]:
def set_key_search(base,j,dir_entry_path,lista=find_lista(),key=find_lista()[0]):
    i=0
    key=key
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json?query=" +str(base[base.columns[0]][j])+"&key="+str(key)
    print(url)
    try:
        respon = requests.get(url)
        print(base.INTFIS_RS_FD_NAME[j])           
        print(json.loads(respon.text)['status'])
        if(json.loads(respon.text)['status']=='ZERO_RESULTS'):
              respon = {}
        else:
            while (json.loads(respon.text)['status']!='OK'):
                key_n=lista[i+1]
                url = "https://maps.googleapis.com/maps/api/place/textsearch/json?query="+str(base.INTFIS_RS_FD_NAME[j])+"&key="+str(key_n)
                respon = requests.get(url)
                i=i+1
                print(json.loads(respon.text)['status'])
                print(base.INTFIS_RS_FD_NAME[j])
    except TimeoutError: 
        print('Timeout')
        time.sleep(10)
        url = "https://maps.googleapis.com/maps/api/place/textsearch/json?query="+str(base.INTFIS_RS_FD_NAME[j])+"&key="+str(key)
        print(base.INTFIS_RS_FD_NAME[j])
        print(json.loads(respon.text)['status'])
    except:
        pass

    return(respon,base[base.columns[0]][j])

In [ ]:
def columna_key(base):
    ind= base[['key']]
    base= base.drop(columns=ind.columns)
    base.insert(loc=0, column=ind.columns[0], value= ind[ind.columns[0]])
    return(base)

In [ ]:
# r1=[]
# def Geocalles(base,dir_entry_path):
#     print(dir_entry_path)
#     dirr=dir_entry_path.replace('.xlsx', '.json')
#     print(dirr)
#     for j in base.index:
#         j=int(j)
#         print(j)
#         resp,base['Domicilio'][0]= set_key(base,j,dirr)
#         try:
#             jj=json.loads(resp.text)
#             y={'key': j}
#             jj.update(y)
#             for result in jj['results']: 
#                 result['key']=j
#             with open(str(dirr), "a", encoding='utf-8') as file:
#                 json.dump(jj, file,ensure_ascii=False)
#         except:
#             print('no se encontró dirección')
#             jj={}
#             y={'key': j}
#             jj.update(y)
        
#             with open(str(dirr), "a", encoding='utf-8') as file:
#                 json.dump(jj, file,ensure_ascii=False)
#     print('Ha terminado el proceso de Json')
#     return(base.Domicilio[j])

In [ ]:
r1=[]
def Geosearch(base,dir_entry_path):
    print(dir_entry_path)
    dirr=dir_entry_path.replace('.xlsx', '.json')
    print(dirr)
    for j in base.index:
        j=int(j)
        print(j)
        resp,base[base.columns[0]][0]= set_key_search(base,j,dirr)
        try:
            jj=json.loads(resp.text)
            y={'key': j}
            jj.update(y)
            for result in jj['results']: 
                result['key']=j
            with open(str(dirr), "a", encoding='utf-8') as file:
                json.dump(jj, file,ensure_ascii=False)
        except:
            print('no se encontró dirección')
            jj={}
            y={'key': j}
            jj.update(y)
        
            with open(str(dirr), "a", encoding='utf-8') as file:
                json.dump(jj, file,ensure_ascii=False)
    print('Ha terminado el proceso de Json')
    return(base[base.columns[0]][j])

In [ ]:
# """"Código de Scraping por calles, utiliza la base Prototipo Segmentada""""
import time
start = time.time()
data={}
path = r""
lista=[]
for dir_entry in os.listdir(path):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):  
        if dir_entry_path.count("xlsx")>0: 
            data[dir_entry] =pd.read_excel(dir_entry_path,sheet_name='Busqueda')
            try:
                B= data[dir_entry]
                #B=B.loc[B['ESQUEMA']!='GCP']
                calle_f=Geosearch(B,dir_entry_path)
                end = time.time()
                print('Ha terminado el proceso para', dir_entry_path)
                print('Tiempo tomado:',end - start)
            except Exception as e:
                raise(e)
                    

In [ ]:
import json
import pandas as pd

In [ ]:
with open(r"", encoding='utf-8-sig') as f:
    file = f.readlines()
    file = ['{'+i+'}'for i in "".join("".join(file).split("\n"))[1:-1].split("}{")]
    CALLES=pd.DataFrame()
    
for i in file:
    try:
        ii=pd.json_normalize(json.loads(i), record_path=['results'],errors='ignore')
        CALLES=pd.concat([CALLES,ii], axis=0)
    except:
        ii= pd.json_normalize(json.loads(i))
        CALLES=pd.concat([CALLES,ii], axis=0)

In [ ]:
CALLES.columns= 'INTFIS_RS_FD_'+CALLES.columns.str.upper()

In [ ]:
CALLES.rename(columns={'INTFIS_RS_FD_GEOMETRY.LOCATION.LAT': 'INTFIS_RS_FD_LAT',
                        'INTFIS_RS_FD_GEOMETRY.LOCATION.LNG': 'INTFIS_RS_FD_LON', 'INTFIS_RS_FD_PLUS_CODE.COMPOUND_CODE': 'INTFIS_RS_FD_PLUS_CODE', 'INTFIS_RS_FD_FORMATTED_ADDRESS':'INTFIS_RS_FD_DIRECCION'}, inplace=True)

In [ ]:
from Geolocalizacion.concatenacion_qr.qr_generation import generate_urls


def creacion_urls(base):
    base = base.reset_index(drop=True)
    base["ORDEN"] = base.index + 1
    # base['INTFIS_RS_FD_LON']=base.geometry.x
    # base['INTFIS_RS_FD_LAT']=base.geometry.y
    # Cambios de formato
    base['LATLON'] = base['INTFIS_RS_FD_LAT'].astype(str).str.cat(base['INTFIS_RS_FD_LON'].astype(str), ' ,')
    complete_section = base[~base.LATLON.isna()]

    assert sum(complete_section.LATLON.isna()) == 0

    complete_section.loc[:, "INTFIS_RS_FD_URLS"] = generate_urls(
        complete_section.LATLON
    )

    base.loc[:, "INTFIS_RS_FD_URLS"] = None
    base = base[base.LATLON.isna()].append(complete_section)

    # Nos aseguramos que todos los URLS tengan un valor de ORDEN unico
    assert len(
        complete_section[(
            ~complete_section.INTFIS_RS_FD_URLS.isna())].ORDEN.unique()
    ) == sum((~complete_section.INTFIS_RS_FD_URLS.isna()))

    # base.drop(columns=["LATLON"], inplace=True)
    # base.to_csv(r'E:\RecepcionInfo\TOTALES_BLOQUE1\Naucalpan de Juarez\Naucalpan_parcial.csv', encoding='utf-8-sig')
    return base